# Feature Engineering

## 목표
- 시계열 특성 생성 (Lag features, Moving averages)
- 추세 및 변동성 지표
- 상대적 지표 (업종 내 순위)
- 고객 행동 지표
- 복합 지표
- Feature Selection

## 작업 내역
1. 전처리 데이터 로드
2. Lag Features (1, 3, 6, 12개월)
3. Moving Averages & Change Rates
4. Trend & Volatility Indicators
5. Relative Indicators (Industry Ranking)
6. Customer Behavior Indicators
7. Composite Indicators
8. Feature Selection
9. Feature Engineering 결과 저장

## 1. 환경 설정

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'DejaVu Sans'

# 출력 설정
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

print("Libraries loaded successfully")

In [ ]:
# 커스텀 모듈 임포트
import sys
sys.path.append('..')

from pipeline.features import (
    TimeSeriesFeatureEngine,
    CustomerFeatureEngine,
    CompositeFeatureEngine
)

print("Custom modules loaded successfully")

## 2. 데이터 로드

In [ ]:
# 전처리된 데이터 로드
data_path = Path('../data/processed/preprocessed_data.csv')
df = pd.read_csv(data_path)

print(f"Loaded data shape: {df.shape}")
print(f"\nColumns: {len(df.columns)}")
print(f"Rows: {len(df):,}")
print(f"Unique merchants: {df['ENCODED_MCT'].nunique():,}")
print(f"Date range: {df['TA_YM'].min()} ~ {df['TA_YM'].max()}")

In [ ]:
# 데이터 미리보기
df.head()

In [ ]:
# 데이터 정보
df.info()

## 3. Lag Features

시계열 데이터에서 과거 값을 현재 시점의 특성으로 사용합니다.
- 1개월 전 (lag_1m)
- 3개월 전 (lag_3m)
- 6개월 전 (lag_6m)
- 12개월 전 (lag_12m)

In [ ]:
# TimeSeriesFeatureEngine 초기화
ts_engine = TimeSeriesFeatureEngine(
    merchant_col='ENCODED_MCT',
    date_col='TA_YM'
)

print("TimeSeriesFeatureEngine initialized")

In [ ]:
# Lag feature를 생성할 주요 변수 선택
lag_columns = [
    # 매출 관련
    'MCT_OPE_MS_CN',  # 가맹점 운영 개월 수
    'RC_M1_SAA',  # 월 매출액
    'M1_SME_RY_SAA_RAT',  # 전월 대비 매출 증감률
    
    # 이용 건수/고객
    'RC_M1_TO_UE_CT',  # 총 이용 건수
    'RC_M1_UE_CUS_CN',  # 이용 고객 수
    'RC_M1_AV_NP_AT',  # 평균 결제 금액
    
    # 고객 재방문/신규
    'MCT_UE_CLN_REU_RAT',  # 재방문 고객 비율
    'MCT_UE_CLN_NEW_RAT',  # 신규 고객 비율
]

print(f"Selected {len(lag_columns)} columns for lag features")
print(f"Columns: {lag_columns}")

In [ ]:
# Lag features 생성 (1, 3, 6, 12개월)
df_with_lags = ts_engine.create_lag_features(
    df,
    columns=lag_columns,
    lags=[1, 3, 6, 12]
)

print(f"\nShape after lag features: {df_with_lags.shape}")
print(f"New columns added: {df_with_lags.shape[1] - df.shape[1]}")

In [ ]:
# Lag feature 샘플 확인
sample_merchant = df_with_lags['ENCODED_MCT'].iloc[0]
sample_cols = ['ENCODED_MCT', 'TA_YM', 'RC_M1_SAA', 'RC_M1_SAA_lag_1m', 'RC_M1_SAA_lag_3m', 'RC_M1_SAA_lag_6m', 'RC_M1_SAA_lag_12m']

print(f"\nSample lag features for merchant {sample_merchant}:")
df_with_lags[df_with_lags['ENCODED_MCT'] == sample_merchant][sample_cols].head(15)

## 4. Moving Averages & Change Rates

추세를 파악하기 위한 이동평균과 증감률을 계산합니다.
- Moving averages: 3개월, 6개월, 12개월
- Change rates: 전월 대비, 전분기 대비, 전년 대비

In [ ]:
# Moving averages 생성할 변수 선택
ma_columns = [
    'RC_M1_SAA',  # 월 매출액
    'RC_M1_TO_UE_CT',  # 총 이용 건수
    'RC_M1_UE_CUS_CN',  # 이용 고객 수
    'RC_M1_AV_NP_AT',  # 평균 결제 금액
    'MCT_UE_CLN_REU_RAT',  # 재방문 고객 비율
]

print(f"Selected {len(ma_columns)} columns for moving averages")

In [ ]:
# Moving averages 생성 (3, 6, 12개월)
df_with_ma = ts_engine.create_moving_averages(
    df_with_lags,
    columns=ma_columns,
    windows=[3, 6, 12]
)

print(f"\nShape after moving averages: {df_with_ma.shape}")
print(f"New columns added: {df_with_ma.shape[1] - df_with_lags.shape[1]}")

In [ ]:
# Change rates 생성 (1개월, 3개월, 12개월 대비)
df_with_change = ts_engine.create_change_rates(
    df_with_ma,
    columns=ma_columns,
    periods=[1, 3, 12]
)

print(f"\nShape after change rates: {df_with_change.shape}")
print(f"New columns added: {df_with_change.shape[1] - df_with_ma.shape[1]}")

In [ ]:
# Moving average & change rate 샘플 확인
sample_cols = [
    'ENCODED_MCT', 'TA_YM', 
    'RC_M1_SAA', 
    'RC_M1_SAA_ma_3m', 'RC_M1_SAA_ma_6m', 'RC_M1_SAA_ma_12m',
    'RC_M1_SAA_change_1m', 'RC_M1_SAA_change_3m', 'RC_M1_SAA_change_12m'
]

available_cols = [col for col in sample_cols if col in df_with_change.columns]

print(f"\nSample moving averages and change rates for merchant {sample_merchant}:")
df_with_change[df_with_change['ENCODED_MCT'] == sample_merchant][available_cols].head(15)

## 5. Trend & Volatility Indicators

가맹점의 안정성과 추세를 나타내는 지표를 생성합니다.
- Trend: 매출 추세 (상승/하락/안정)
- Volatility: 매출 변동성 (표준편차, 변동계수)

In [ ]:
# Trend & Volatility 지표 생성할 변수
trend_columns = [
    'RC_M1_SAA',  # 월 매출액
    'RC_M1_TO_UE_CT',  # 총 이용 건수
    'RC_M1_UE_CUS_CN',  # 이용 고객 수
]

print(f"Selected {len(trend_columns)} columns for trend & volatility")

In [ ]:
# Trend indicators 생성 (3, 6, 12개월)
df_with_trend = ts_engine.create_trend_indicators(
    df_with_change,
    columns=trend_columns,
    windows=[3, 6, 12]
)

print(f"\nShape after trend indicators: {df_with_trend.shape}")
print(f"New columns added: {df_with_trend.shape[1] - df_with_change.shape[1]}")

In [ ]:
# Volatility indicators 생성 (3, 6, 12개월)
df_with_volatility = ts_engine.create_volatility_indicators(
    df_with_trend,
    columns=trend_columns,
    windows=[3, 6, 12]
)

print(f"\nShape after volatility indicators: {df_with_volatility.shape}")
print(f"New columns added: {df_with_volatility.shape[1] - df_with_trend.shape[1]}")

In [ ]:
# Trend & Volatility 샘플 확인
sample_cols = [
    'ENCODED_MCT', 'TA_YM',
    'RC_M1_SAA',
    'RC_M1_SAA_trend_3m', 'RC_M1_SAA_trend_6m', 'RC_M1_SAA_trend_12m',
    'RC_M1_SAA_std_3m', 'RC_M1_SAA_std_6m', 'RC_M1_SAA_std_12m',
    'RC_M1_SAA_cv_3m', 'RC_M1_SAA_cv_6m', 'RC_M1_SAA_cv_12m'
]

available_cols = [col for col in sample_cols if col in df_with_volatility.columns]

print(f"\nSample trend & volatility indicators for merchant {sample_merchant}:")
df_with_volatility[df_with_volatility['ENCODED_MCT'] == sample_merchant][available_cols].head(15)

## 6. Relative Indicators (Industry Ranking)

업종 내에서의 상대적 위치를 나타내는 지표를 생성합니다.
- 월별 매출 순위
- 순위 변화 추이
- 업종 평균 대비 비율

In [ ]:
# Relative ranking 지표 생성
ranking_columns = [
    'RC_M1_SAA',  # 월 매출액
    'RC_M1_TO_UE_CT',  # 총 이용 건수
    'RC_M1_UE_CUS_CN',  # 이용 고객 수
]

print(f"Selected {len(ranking_columns)} columns for relative ranking")

In [ ]:
# Ranking indicators 생성 (월별 순위)
df_with_ranking = ts_engine.create_ranking_indicators(
    df_with_volatility,
    columns=ranking_columns
)

print(f"\nShape after ranking indicators: {df_with_ranking.shape}")
print(f"New columns added: {df_with_ranking.shape[1] - df_with_volatility.shape[1]}")

In [ ]:
# Ranking change 생성 (순위 변화)
df_with_rank_change = ts_engine.create_ranking_change(
    df_with_ranking,
    columns=ranking_columns,
    periods=[1, 3, 6]
)

print(f"\nShape after ranking change: {df_with_rank_change.shape}")
print(f"New columns added: {df_with_rank_change.shape[1] - df_with_ranking.shape[1]}")

In [ ]:
# Ranking 샘플 확인
sample_cols = [
    'ENCODED_MCT', 'TA_YM',
    'RC_M1_SAA',
    'RC_M1_SAA_rank', 'RC_M1_SAA_rank_pct',
    'RC_M1_SAA_rank_change_1m', 'RC_M1_SAA_rank_change_3m', 'RC_M1_SAA_rank_change_6m'
]

available_cols = [col for col in sample_cols if col in df_with_rank_change.columns]

print(f"\nSample ranking indicators for merchant {sample_merchant}:")
df_with_rank_change[df_with_rank_change['ENCODED_MCT'] == sample_merchant][available_cols].head(15)

## 7. Customer Behavior Indicators

고객 행동 패턴을 나타내는 지표를 생성합니다.
- 재방문율 추이
- 신규 고객 비율 추이
- 고객 충성도 지표

In [ ]:
# CustomerFeatureEngine 초기화
customer_engine = CustomerFeatureEngine(
    merchant_col='ENCODED_MCT',
    date_col='TA_YM'
)

print("CustomerFeatureEngine initialized")

In [ ]:
# Customer behavior indicators 생성
df_with_customer = customer_engine.create_customer_behavior_features(
    df_with_rank_change,
    windows=[3, 6, 12]
)

print(f"\nShape after customer behavior features: {df_with_customer.shape}")
print(f"New columns added: {df_with_customer.shape[1] - df_with_rank_change.shape[1]}")

In [ ]:
# Customer loyalty indicators 생성
df_with_loyalty = customer_engine.create_loyalty_indicators(
    df_with_customer,
    windows=[3, 6, 12]
)

print(f"\nShape after loyalty indicators: {df_with_loyalty.shape}")
print(f"New columns added: {df_with_loyalty.shape[1] - df_with_customer.shape[1]}")

## 8. Composite Indicators

여러 feature를 조합하여 복합 지표를 생성합니다.
- 가맹점 건강도 지수
- 리스크 지수
- 성장 지수

In [ ]:
# CompositeFeatureEngine 초기화
composite_engine = CompositeFeatureEngine()

print("CompositeFeatureEngine initialized")

In [ ]:
# Composite indicators 생성
df_with_composite = composite_engine.create_composite_indicators(
    df_with_loyalty
)

print(f"\nShape after composite indicators: {df_with_composite.shape}")
print(f"New columns added: {df_with_composite.shape[1] - df_with_loyalty.shape[1]}")

## 9. Feature Selection

생성된 feature 중 중요한 변수를 선별합니다.
- 결측값 비율 확인
- 분산 확인
- 상관관계 분석

In [ ]:
# 결측값 비율 확인
missing_ratio = df_with_composite.isnull().sum() / len(df_with_composite) * 100
missing_ratio = missing_ratio[missing_ratio > 0].sort_values(ascending=False)

print(f"Columns with missing values: {len(missing_ratio)}")
print(f"\nTop 20 columns with highest missing ratio:")
print(missing_ratio.head(20))

In [ ]:
# 결측값 시각화
if len(missing_ratio) > 0:
    fig, ax = plt.subplots(figsize=(14, 8))
    top_missing = missing_ratio.head(30)
    ax.barh(range(len(top_missing)), top_missing.values)
    ax.set_yticks(range(len(top_missing)))
    ax.set_yticklabels(top_missing.index)
    ax.set_xlabel('Missing Ratio (%)')
    ax.set_title('Top 30 Columns with Missing Values')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()

In [ ]:
# 수치형 변수 선택
numeric_cols = df_with_composite.select_dtypes(include=[np.number]).columns.tolist()

# ID, 타겟 변수 제외
exclude_cols = ['ENCODED_MCT', 'TA_YM', 'is_closed', 'will_close_1m', 'will_close_3m', 'months_until_close']
feature_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"\nTotal numeric features: {len(feature_cols)}")
print(f"Features: {feature_cols[:20]}...")

In [ ]:
# 분산이 0인 변수 확인
zero_var_cols = df_with_composite[feature_cols].var()[df_with_composite[feature_cols].var() == 0].index.tolist()

if zero_var_cols:
    print(f"\nColumns with zero variance: {len(zero_var_cols)}")
    print(zero_var_cols)
else:
    print("\nNo columns with zero variance found")

In [ ]:
# 타겟 변수와의 상관관계 확인 (is_closed)
if 'is_closed' in df_with_composite.columns:
    correlations = df_with_composite[feature_cols].corrwith(df_with_composite['is_closed']).abs().sort_values(ascending=False)
    
    print("\nTop 30 features correlated with 'is_closed':")
    print(correlations.head(30))
    
    # 시각화
    fig, ax = plt.subplots(figsize=(14, 8))
    top_corr = correlations.head(30)
    ax.barh(range(len(top_corr)), top_corr.values)
    ax.set_yticks(range(len(top_corr)))
    ax.set_yticklabels(top_corr.index)
    ax.set_xlabel('Absolute Correlation with is_closed')
    ax.set_title('Top 30 Features by Correlation with Target')
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()

## 10. Feature Engineering 결과 저장

In [ ]:
# 최종 데이터 정보
print("="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)

print(f"\nFinal Shape: {df_with_composite.shape}")
print(f"Total Columns: {len(df_with_composite.columns)}")
print(f"Total Rows: {len(df_with_composite):,}")
print(f"Unique Merchants: {df_with_composite['ENCODED_MCT'].nunique():,}")
print(f"Date Range: {df_with_composite['TA_YM'].min()} ~ {df_with_composite['TA_YM'].max()}")
print(f"Memory Usage: {df_with_composite.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nOriginal columns: {df.shape[1]}")
print(f"Engineered features: {df_with_composite.shape[1] - df.shape[1]}")
print(f"Missing Values: {df_with_composite.isnull().sum().sum():,}")

In [ ]:
# Feature engineering 데이터 저장
output_path = Path('../data/processed')
output_path.mkdir(exist_ok=True)

# CSV 저장
csv_path = output_path / 'featured_data.csv'
df_with_composite.to_csv(csv_path, index=False)
print(f"\nSaved to: {csv_path}")
print(f"File size: {csv_path.stat().st_size / 1024**2:.2f} MB")

In [ ]:
# Feature list 저장
feature_list = {
    'total_features': len(feature_cols),
    'feature_columns': feature_cols,
    'lag_features': [col for col in df_with_composite.columns if '_lag_' in col],
    'ma_features': [col for col in df_with_composite.columns if '_ma_' in col],
    'change_features': [col for col in df_with_composite.columns if '_change_' in col],
    'trend_features': [col for col in df_with_composite.columns if '_trend_' in col],
    'volatility_features': [col for col in df_with_composite.columns if '_std_' in col or '_cv_' in col],
    'ranking_features': [col for col in df_with_composite.columns if '_rank' in col],
    'customer_features': [col for col in df_with_composite.columns if 'customer_' in col or 'loyalty_' in col],
    'composite_features': [col for col in df_with_composite.columns if 'health_' in col or 'risk_' in col or 'growth_' in col],
}

import json
feature_path = output_path / 'feature_list.json'
with open(feature_path, 'w') as f:
    json.dump(feature_list, f, indent=2)

print(f"\nFeature list saved to: {feature_path}")

In [ ]:
# Feature 통계 요약
print("\nFeature Statistics:")
for key, value in feature_list.items():
    if key != 'feature_columns':
        if isinstance(value, list):
            print(f"{key}: {len(value)}")
        else:
            print(f"{key}: {value}")

## 11. 결론

### 완료된 작업
1. ✅ Lag Features 생성 (1, 3, 6, 12개월)
2. ✅ Moving Averages & Change Rates
3. ✅ Trend & Volatility Indicators
4. ✅ Relative Indicators (Ranking)
5. ✅ Customer Behavior Indicators
6. ✅ Composite Indicators
7. ✅ Feature Selection & Analysis
8. ✅ Feature Engineering 결과 저장

### 다음 단계 (Week 3)
- 모델 학습 및 평가
  - XGBoost, LightGBM 모델 학습
  - Cross-validation
  - Hyperparameter tuning
  - Feature importance 분석
  - SHAP value 분석
  - 앙상블 모델

In [ ]:
print("\n" + "="*80)
print("FEATURE ENGINEERING COMPLETED")
print("="*80)
print(f"\nFinal dataset: {df_with_composite.shape}")
print(f"Saved to: {csv_path}")
print("\nReady for Model Training!")